# Gene set and pathway

![](./images/Module3/Workflow3.jpg)

Differential expression (DE) analysis typically yields a list of genes or proteins that require further interpretation.
Our intention is to use such lists to gain novel insights about genes and proteins that may have roles in a given
phenomenon, phenotype or disease progression. However, in many cases, gene lists generated from DE analysis are
difficult to interpret due to their large size and lack of useful annotations. Hence, pathway analysis (also known
as gene set analysis or over-representation analysis), aims to reduce the complexity of interpreting gene lists
via mapping the listed genes to known (i.e. annotated) biological pathways, processes and functions.
This learning module introduces common curated biological databases including Gene Ontology (GO),
Kyoto Encyclopedia of Genes and Genomes (KEGG).

## Learning Objectives:
1. Introduction to Ontology and Gene Ontology.
2. Introduction to KEGG Pathway Database.
3. Downloading terms, pathway gene set from GO and KEGG.
4. Saving results to GMT file format.

## Ontology and Gene Ontology
### Overview
In this section we will learn about the concept of gene ontology in Bioinformatics. Ontology is a specification
of a conceptualization which is a set of concepts within a domain, defined by a shared vocabulary to
denote the types and properties of the concepts as well as the relationships between the concepts.
Ontology plays an important role in the field of bioinformatics. Ontology enable communicate unambiguously e.g.,
to understand across different groups’ annotations of various genomes. Also, it allows the knowledge to be represented
in a computable and structured to perform automated analyses by computer programs.

The Gene Ontology (GO) defines a structured, common, controlled vocabulary to describe attributes of genes and gene products
across organisms. Collaboration is key to build a consensus vocabulary. But the term gene ontology, or GO, is commonly used
to refer to both, which is sometimes a source of potential confusion. In order to avoid this, here we will use the term “GO”
to describe the set of terms and their hierarchical structure and “GO annotations” to describe the set of associations between
genes and GO terms. The GO is divided into three categories to describe the genes and gene products from three different
angles: Molecular Function, Biological Process, and Cellular Component.

The structure of GO can be described in terms of into directed acyclic graphs (DAGs), where each GO term is a node,
and the relationships between the terms are edges between the nodes. GO is loosely hierarchical, with ‘child’ terms
being more specialized than their ‘parent’ terms, but unlike a strict hierarchy, a term may have more than one parent
term (note that the parent/child model does not hold true for all types of relations). The structure of the controlled
vocabularies are intended to reflect true, biological relationships. In contrast to strict hierarchies, DAGS allow
multiple relationships between a more granular (child) term and a more general parent term. The relationship between
terms affects how queries are made. For example,a query for all genes with binding activity would include transcription
factors as well as genes with other types of binding activity (such as protein binding, ligand binding). The illustration
of category and structure of GO is shown in the figure below:

![](./images/Module3/GO_Structure.jpg)
*(Source: https://www.ebi.ac.uk/, http://geneontology.org/)*

### Gene ontology relationship
In DAGs graph, *terms* are represented as *nodes* and *relations* (also known as *object properties*) between the *terms*
are *edges*. There are commonly used relationships in GO such as *is a* (is a *subtype of*), *part of, has part, regulates,
negatively regulates and positively regulates*. All terms (except from the root terms representing each aspect) have an
is a sub-class relationship to another terms.

Examples:

```{admonition} *is a* relation
**GO:1904659:glucose transport** *is* a **GO:0015749:monosaccharide transport**.

The *is a* relation forms the basic structure of GO. If we say A *is a* B, we mean that node A is a subtype of node B

```

```{admonition} *is part of* relation
**GO:0031966:mitochondrial membrane** *is part of* **GO:0005740:mitochondrial envelope**

The *part of* relation is used to represent part-whole relationships. A *part of* relation would only be added between
A and B if B is **necessarily** *part of* A: wherever B exists, it is as *part of* A, and the presence of the B implies
the presence of A. However, given the occurrence of A, we cannot say for certain that B exists.
```

```{admonition} *regulates* relation
**GO:0098689:latency-replication decision** *regulates* **GO:0019046:release from viral latency**

A relation that describes case in which one process directly affects the manifestation of another process or quality,
i.e. the former *regulates* the latter.
```

A more specific case with more nodes and edges can be seen at the figure below:
<br>
![](./images/Module3/GO_Relation.jpg)
*(Source: https://advaitabio.com/)* <br>
For more technical information about relations and their properties used in GO and other ontologies see the
<a href="https://obofoundry.org/ontology/ro.html">OBO Relations Ontology (RO)</a>


### GO storage file formats
GO terms are updated monthly in the following formats:
* OBO 1.4 files are human-readable (in addition to machine-readable) and can be opened in any text editor.
* OWL files can be read by <a href="https://protege.stanford.edu/">Protégé</a> text editor.

 In this learning module, we will only use ".OBO" to obtain GO terms.The OBO file format is for representing ontologies and controlled vocabularies. The format itself attempts to achieve the following goals:
 * Human readability
 * Ease of parsing
 * Extensibility
 * Minimal redundancy

The file structure can is shown in the following figure.


![](./images/Module3/OBO_Format.jpg)

The OBO file has a header which is an unlabeled section at the beginning of the document. The header ends when the first term is encountered. Next, term is represented in labeled section with the tag *[Term]*. Under each term, we can find other information such as term ID, official name, category (namcespace), term definition, synonym and relation to other GO term.

At this step, we still don't know what genes are related to which GO terms. In order to retrieve custom sets of gene ontology annotations for any list of genes from organisms, NCBI has published a Gene2GO database that obtain GO terms and the entrez gene ids related to those go terms. The database can be retrieve from <a href="https://ftp.ncbi.nih.gov/gene/DATA/gene2go.gz">here</a> text editor. The Gene2GO database can be viewed using text editor, the file structure is presented in the figure below:

![](./images/Module3/Gene2GO.jpg)

The OBO and Gene2GO databases will be used in combination to obtain GO term and related genes for enrichment analysis.

### Retrieving GO terms from DE gene list


In [4]:
suppressMessages({if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
  suppressWarnings(BiocManager::install("topGO", update = F))
  suppressWarnings(BiocManager::install("hgu133plus2.db", update = F))
})

In [12]:
suppressPackageStartupMessages({
  library("topGO")
  library("hgu133plus2.db")
})

In [27]:
data = readRDS("./data/DE_genes.rds")

In [28]:
gl <- data$adj.P.Val
names(gl) <- data$ID

## we need a vector like yours, so do some stuff
z <- select(hgu133plus2.db, names(gl), "SYMBOL")
z <- z[!duplicated(z[,1]),]
geneList2 <- gl
names(geneList2) <- z[,2]

sampleGOdata <- new("topGOdata", description = "",ontology = "BP",
                    allGenes = geneList2, geneSel = function(x)x, nodeSize = 10,
                    annot = annFUN.org, ID = "alias", mapping = "org.Hs.eg")


allGO = genesInTerm(sampleGOdata)

'select()' returned 1:many mapping between keys and columns


Building most specific GOs .....

	( 2203 GO terms found. )


Build GO DAG topology ..........

	( 5410 GO terms and 11936 relations. )


Annotating nodes ...............

	( 478 genes annotated to the GO terms. )



In [29]:
allGO[1:5]

$`GO:0000003`
 [1] "ALDOA"   "ALKBH1"  "ARID5B"  "ATP2B2"  "B4GALT1" "BRF1"    "CACNA1H"
 [8] "CCDC40"  "CCNB1"   "CCNE1"   "CD9"     "CFAP157" "CFAP54"  "CYLC1"  
[15] "EHMT2"   "ESR1"    "FAM170A" "FKBP6"   "FOLR2"   "FUT10"   "GAMT"   
[22] "HGF"     "HYAL3"   "IGFBP7"  "LY6K"    "MYCBPAP" "OAZ3"    "OVOL1"  
[29] "PCDH12"  "QRICH2"  "SIX5"    "SLX4"    "SPATA6L" "ST14"    "STK11"  
[36] "STK35"   "TAC1"    "TEKT2"   "TLE6"    "TMEM95"  "TRIM27"  "YBX2"   

$`GO:0000122`
 [1] "ARID5B"  "BARX2"   "CCNE1"   "CTBP1"   "CUX2"    "EHMT2"   "ESR1"   
 [8] "GLIS3"   "GRIP1"   "HIF1AN"  "HOPX"    "HOXA7"   "LMO1"    "NR2F6"  
[15] "OTUD7B"  "OVOL1"   "RTF1"    "RUNX3"   "SP100"   "SPDEF"   "SREBF1" 
[22] "TAF1"    "TAL1"    "TGIF2"   "TLE6"    "TMPRSS6" "TPR"     "TRIM27" 
[29] "TRIM37"  "WWC2"    "WWTR1"   "ZBTB45"  "ZC3H8"   "ZNF345" 

$`GO:0000165`
 [1] "ADRB3"    "BRF1"     "CCL22"    "CLEC7A"   "COPS5"    "FGD2"    
 [7] "FGF18"    "FGF4"     "HGF"      "LILRA5"   "MAP4K1"   "NCF1"    
[13] "NENF"     "P2RY6"    "PRDX2"    "PRMT1"    "RAP2A"    "RB1CC1"  
[19] "SERPINF2" "SLAMF1"   "STYX"     "TEK"      "TNIP1"    "WNK2"    

$`GO:0000226`
 [1] "ANKFN1"   "ARHGEF7"  "ATF5"     "CCDC40"   "CCNB1"    "CDC14A"  
 [7] "CFAP100"  "CFAP157"  "CNTROB"   "DCLK2"    "DCTN1"    "DNAAF8"  
[13] "DST"      "GAS2L2"   "RAE1"     "TACC3"    "TEKT2"    "TLE6"    
[19] "TPR"      "TRAF3IP1" "TRIM37"   "TUBB4B"   "ZMYND10"  "ZPR1"    

$`GO:0000278`
 [1] "ANKFN1" "BRF1"   "CCNB1"  "CCNE1"  "CDC14A" "CDC5L"  "CNTROB" "DCTN1" 
 [9] "DCTN3"  "DSCC1"  "FBXL17" "GSPT1"  "HASPIN" "HGF"    "OVOL1"  "PBRM1" 
[17] "PML"    "RAE1"   "SMC5"   "SPTBN1" "TACC3"  "TAL1"   "TPR"    "TRIAP1"
[25] "TUBB4B" "VCPIP1" "ZPR1"

In [31]:
library(GO.db)

In [35]:
writeGMT <- function(genesets, descriptions, outfile) {

  if (file.exists(outfile)) {
    message(paste(outfile, "exists, removing..."))
    file.remove(outfile)
  }
  for (gs in names(genesets)) {
    write(c(gs, gsub("\t", " ", descriptions[[gs]]), genesets[[gs]]), file=outfile, sep="\t", append=TRUE, ncolumns=length(genesets[[gs]]) + 2)
  }
}

In [33]:
terms <- names(allGO)
descriptions <-lapply(Term(terms), `[[`, 1)

In [37]:
outfile <- "./data/GO_terms.gmt"
writeGMT(allGO, descriptions, outfile)

./data/GO_terms.gmt exists, removing...



## Kyoto Encyclopedia of Genes and Genomes (KEGG)
### Overview
KEGG is a collection of databases dealing with genomes, biological pathways, diseases, drugs, and chemical substances. KEGG is utilized for bioinformatics research and education, including data analysis in genomics, metagenomics, metabolomics and other omics studies, modeling and simulation in systems biology, and translational research in drug development. The KEGG database project was initiated in 1995 by Minoru Kanehisa, professor at the Institute for Chemical Research, Kyoto University, under the then ongoing Japanese Human Genome Program. Foreseeing the need for a computerized resource that can be used for biological interpretation of genome sequence data, he started developing the KEGG PATHWAY database. It is a collection of manually drawn KEGG pathway maps representing experimental knowledge on metabolism and various other functions of the cell and the organism. Each pathway map contains a network of molecular interactions and reactions and is designed to link genes in the genome to gene products (mostly proteins) in the pathway. This has enabled the analysis called KEGG pathway mapping, whereby the gene content in the genome is compared with the KEGG PATHWAY database to examine which pathways and associated functions are likely to be encoded in the genome. KEGG is a "computer representation" of the biological system. It integrates building blocks and wiring diagrams of the system—more specifically, genetic building blocks of genes and proteins, chemical building blocks of small molecules and reactions, and wiring diagrams of molecular interaction and reaction networks. The illustrative structure of KEGG is presented as figure below.
![](./images/Module3/KEGG.jpg)


This concept is realized in the following databases of KEGG, which are categorized into systems, genomic, chemical, and health information.
